# Day 8: Assistant Agent - Your Production-Ready AI Partner

## Welcome to the Power Tool!

Today you'll master the **Assistant** - the most complete and production-ready agent in Qwen-Agent.

**What you've learned so far:**
- Day 4: Tools give agents real capabilities
- Day 5: Custom agents with `_run()` method
- Day 6: Function calling for tool selection
- Day 7: Creating custom tools with `@register_tool`

**Today: The Assistant brings it ALL together!**

The Assistant agent is like hiring a super-capable employee who:
- ✅ Knows how to use tools automatically
- ✅ Can read and understand documents (RAG)
- ✅ Follows your instructions precisely
- ✅ Handles errors gracefully
- ✅ Streams responses in real-time
- ✅ Maintains conversation context

### Today's Journey:
1. **All initialization parameters** - Master every option
2. **files parameter magic** - Automatic document knowledge
3. **System message engineering** - Control agent behavior
4. **function_list flexibility** - Tools in multiple formats
5. **Real-world examples** - Customer support, code helper, analyst
6. **Production patterns** - Error handling, streaming, memory

Let's build production-ready assistants! 🚀

---
## Part 1: Setup

Same Fireworks API configuration as previous days.

In [ ]:
import os
import json
import json5

os.environ['FIREWORKS_API_KEY'] = 'fw_3ZSpUnVR78vs38jJtyewjcWk'

llm_cfg = {
    'model': 'accounts/fireworks/models/qwen3-235b-a22b-thinking-2507',
    'model_server': 'https://api.fireworks.ai/inference/v1',
    'api_key': os.environ['FIREWORKS_API_KEY'],
    'generate_cfg': {'max_tokens': 32768, 'temperature': 0.6}
}

print('✅ Fireworks API configured')

---
## Part 2: Assistant Parameters - Complete Reference

### The Full Signature

```python
Assistant(
    llm,                    # Required: LLM configuration
    function_list=None,     # Optional: List of tools
    name=None,              # Optional: Agent's name
    description=None,       # Optional: What agent does
    system_message=None,    # Optional: Behavior instructions
    files=None              # Optional: Documents for RAG
)
```

### Parameter Guide:

| Parameter | Required? | Type | Purpose |
|-----------|-----------|------|--------|
| `llm` | ✅ Yes | Dict or BaseChatModel | The AI brain |
| `function_list` | ❌ No | List | Tools the agent can use |
| `name` | ❌ No | String | Agent's identity |
| `description` | ❌ No | String | What the agent does |
| `system_message` | ❌ No | String | Behavior/personality |
| `files` | ❌ No | List[str] | Documents to learn from |

Let's explore each parameter with examples!

### Example 1: Minimal Assistant (Just LLM)

The simplest possible assistant - just an LLM, no tools, no files.

In [ ]:
from qwen_agent.agents import Assistant

# Minimal assistant - just LLM
basic_bot = Assistant(llm=llm_cfg)

# Test it
messages = [{'role': 'user', 'content': 'Explain quantum computing in one sentence'}]

print("User: Explain quantum computing in one sentence\n")
for response in basic_bot.run(messages):
    if response:
        print(f"Assistant: {response[-1].get('content', '')}\n")

### Example 2: Assistant with Name and Description

Giving your assistant an identity helps with logging and debugging.

In [ ]:
# Assistant with identity
named_bot = Assistant(
    llm=llm_cfg,
    name='PhysicsExpert',
    description='Expert in physics and mathematics'
)

print(f"Bot name: {named_bot.name}")
print(f"Bot description: {named_bot.description}\n")

# The name/description are mainly for logging and organization
messages = [{'role': 'user', 'content': 'What is entropy?'}]
for response in named_bot.run(messages):
    if response:
        print(f"{named_bot.name}: {response[-1].get('content', '')[:200]}...\n")

---
## Part 3: System Message Engineering

### What is system_message?

The `system_message` is like **giving instructions to an employee** on their first day.

It controls:
- ✅ Personality and tone
- ✅ Response format
- ✅ Expertise and knowledge
- ✅ Behavior rules
- ✅ When to use tools

Let's see the power of system messages!

### Example 3: Role-Playing with System Message

In [ ]:
# Pirate assistant
pirate_bot = Assistant(
    llm=llm_cfg,
    name='CaptainCodebeard',
    system_message="""You are a friendly pirate captain who teaches programming.
Always:
- Talk like a pirate (use 'arr', 'matey', 'ye')
- Make sailing/ocean analogies
- Be enthusiastic and encouraging
- Keep technical accuracy"""
)

messages = [{'role': 'user', 'content': 'Explain what a Python function is'}]

print("🏴‍☠️ Pirate Programming Tutor:\n")
for response in pirate_bot.run(messages):
    if response:
        print(response[-1].get('content', ''))

### Example 4: Output Formatting with System Message

In [ ]:
# JSON output assistant
json_bot = Assistant(
    llm=llm_cfg,
    system_message="""You are a data extraction assistant.
ALWAYS respond in this JSON format:
{
  "summary": "brief summary",
  "key_points": ["point 1", "point 2"],
  "confidence": 0.95
}

No other text outside the JSON."""
)

messages = [{'role': 'user', 'content': 'Summarize the benefits of exercise'}]

print("Structured JSON Output:\n")
for response in json_bot.run(messages):
    if response:
        print(response[-1].get('content', ''))

### Example 5: Expert Persona System Message

In [ ]:
# Medical expert (educational purposes only)
medical_bot = Assistant(
    llm=llm_cfg,
    name='MedicalEducator',
    system_message="""You are a medical educator for students.

Guidelines:
- Explain medical concepts clearly
- Use analogies for complex topics
- Always include disclaimers for health advice
- Cite general medical knowledge
- Encourage consulting real doctors

Format:
1. Main explanation
2. Example or analogy
3. Disclaimer"""
)

messages = [{'role': 'user', 'content': 'How does insulin work?'}]

print("Medical Education Mode:\n")
for response in medical_bot.run(messages):
    if response:
        print(response[-1].get('content', '')[:400] + '...\n')

---
## Part 4: The files Parameter - Automatic RAG

### What is RAG?

**RAG (Retrieval-Augmented Generation)** means:
1. Give your agent documents to learn from
2. When user asks a question, agent finds relevant parts
3. Agent uses those parts to answer accurately

### How files Works

When you provide `files` parameter:
```python
Assistant(llm=llm_cfg, files=['document.pdf'])
```

Qwen-Agent automatically:
1. ✅ Reads the document
2. ✅ Splits into chunks
3. ✅ Creates embeddings
4. ✅ Retrieves relevant chunks for each query
5. ✅ Augments LLM context

**You get instant document Q&A!**

### Example 6: RAG with Local File

In [ ]:
# Create a sample company policy document
policy_content = """ACME Corporation Employee Handbook

1. VACATION POLICY
Employees receive 20 vacation days per year.
Vacation must be requested 2 weeks in advance.
Unused vacation carries over up to 5 days.

2. REMOTE WORK POLICY
Employees may work remotely 3 days per week.
Core hours (10 AM - 3 PM) must be available online.
Monthly in-office meetings are mandatory.

3. SICK LEAVE
Unlimited sick leave with doctor's note.
First 3 days don't require documentation.
Notify manager by 9 AM.

4. PROFESSIONAL DEVELOPMENT
$2000 annual budget for courses/conferences.
Must be job-related and pre-approved.
Time off granted for attending.

5. EQUIPMENT
Company provides laptop and monitor.
Home office stipend: $500 annually.
IT support available 24/7.
"""

# Save to file
with open('acme_policy.txt', 'w') as f:
    f.write(policy_content)

print("✅ Created company policy document")

# Create RAG-enabled assistant
hr_assistant = Assistant(
    llm=llm_cfg,
    name='HR Assistant',
    description='Helps employees understand company policies',
    system_message="""You are a friendly HR assistant.

Guidelines:
- Answer based on the company policy document
- Be specific and cite policy sections
- If information isn't in the document, say so
- Be helpful and friendly""",
    files=[os.path.abspath('acme_policy.txt')]
)

print("✅ Created HR Assistant with RAG\n")

In [ ]:
# Test the RAG assistant with various questions
test_questions = [
    "How many vacation days do I get?",
    "What's the remote work policy?",
    "Do I get money for professional development?",
    "What happens if I'm sick?"
]

for question in test_questions:
    print(f"\n{'='*60}")
    print(f"User: {question}")
    print(f"{'='*60}\n")
    
    messages = [{'role': 'user', 'content': question}]
    
    for response in hr_assistant.run(messages):
        if response:
            answer = response[-1].get('content', '')
            print(f"HR Assistant: {answer}\n")

### Example 7: RAG with URL (Arxiv Paper)

In [ ]:
# Research paper assistant (from official examples)
research_bot = Assistant(
    llm=llm_cfg,
    name='Research Assistant',
    system_message='You help researchers understand academic papers. Be technical but clear.',
    files=['https://arxiv.org/pdf/1706.03762.pdf']  # Famous "Attention Is All You Need" paper
)

# Ask about the paper
messages = [{'role': 'user', 'content': 'What is the main contribution of this paper?'}]

print("Research Assistant (reading Transformer paper):\n")
for response in research_bot.run(messages):
    if response:
        print(response[-1].get('content', '')[:300] + '...\n')

### File Handling in Messages

You can also pass files directly in messages (in addition to or instead of files parameter):

In [ ]:
# Assistant without pre-loaded files
flexible_bot = Assistant(llm=llm_cfg)

# Pass file in the message itself
messages = [{
    'role': 'user',
    'content': [
        {'text': 'What is the vacation policy in this document?'},
        {'file': os.path.abspath('acme_policy.txt')}
    ]
}]

print("File passed in message:\n")
for response in flexible_bot.run(messages):
    if response:
        print(response[-1].get('content', '')[:200] + '...\n')

---
## Part 5: function_list - Tool Integration

### Multiple Ways to Provide Tools

The `function_list` parameter accepts:
1. ✅ String names (built-in tools)
2. ✅ Tool configuration dicts
3. ✅ BaseTool instances (custom tools)
4. ✅ MCP server configs
5. ✅ Mixed combinations

Let's see each!

### Example 8: String Names (Built-in Tools)

In [ ]:
# Assistant with code execution capability
code_bot = Assistant(
    llm=llm_cfg,
    function_list=['code_interpreter'],  # String name
    system_message='You are a helpful coding assistant. Use code execution when needed.'
)

messages = [{'role': 'user', 'content': 'Calculate the factorial of 15'}]

print("Code execution assistant:\n")
for response in code_bot.run(messages):
    for msg in response:
        if msg.get('function_call'):
            print(f"🔧 Calling: {msg['function_call']['name']}\n")
        elif msg.get('content'):
            print(f"Response: {msg['content']}\n")

### Example 9: Custom Tool Instances

In [ ]:
from qwen_agent.tools.base import BaseTool, register_tool
import urllib.parse

# Create custom tool
@register_tool('weather_api')
class WeatherTool(BaseTool):
    description = 'Get current weather for a city'
    parameters = [{
        'name': 'city',
        'type': 'string',
        'description': 'City name',
        'required': True
    }]
    
    def call(self, params, **kwargs):
        city = json5.loads(params)['city']
        # Simulated response
        return json.dumps({'city': city, 'temp': 72, 'condition': 'Sunny'})

# Use custom tool
weather_bot = Assistant(
    llm=llm_cfg,
    function_list=['weather_api'],  # Our custom tool
    system_message='You are a weather assistant. Always use the weather tool to get current conditions.'
)

messages = [{'role': 'user', 'content': 'What\'s the weather in Tokyo?'}]

print("Weather assistant:\n")
for response in weather_bot.run(messages):
    for msg in response:
        if msg.get('content'):
            print(msg['content'])

### Example 10: Mixed Tools (Built-in + Custom)

In [ ]:
# Assistant with multiple tool types
multi_tool_bot = Assistant(
    llm=llm_cfg,
    function_list=[
        'code_interpreter',  # Built-in
        'weather_api'        # Custom
    ],
    system_message='You are a versatile assistant. Use appropriate tools for each task.'
)

test_queries = [
    "Calculate 2^100",
    "What's the weather in London?"
]

for query in test_queries:
    print(f"\n{'='*60}")
    print(f"User: {query}")
    print(f"{'='*60}\n")
    
    messages = [{'role': 'user', 'content': query}]
    
    for response in multi_tool_bot.run(messages):
        for msg in response:
            if msg.get('function_call'):
                print(f"→ Using tool: {msg['function_call']['name']}")
            elif msg.get('content'):
                print(f"→ {msg['content'][:150]}\n")

---
## Part 6: Real-World Example - Customer Support Bot

Let's build a production-ready customer support assistant that combines everything!

In [ ]:
# Create product FAQ document
faq_content = """Product FAQ - SmartWatch Pro

Q: What is the battery life?
A: Up to 7 days with normal use, 3 days with GPS active.

Q: Is it waterproof?
A: Yes, rated IP68. Safe for swimming and showering.

Q: What phones does it work with?
A: Compatible with iPhone (iOS 13+) and Android (8.0+).

Q: How do I charge it?
A: Use included magnetic charging cable. Full charge takes 2 hours.

Q: Can I make calls?
A: Yes, with built-in speaker and microphone via Bluetooth.

Q: What's the return policy?
A: 30-day money-back guarantee. Free returns.

Q: What sensors does it have?
A: Heart rate, SpO2, accelerometer, gyroscope, GPS, altimeter.
"""

with open('product_faq.txt', 'w') as f:
    f.write(faq_content)

print("✅ Created product FAQ")

# Build comprehensive support bot
support_bot = Assistant(
    llm=llm_cfg,
    name='SmartWatch Support',
    description='Customer support for SmartWatch Pro products',
    system_message="""You are a friendly and professional customer support agent for SmartWatch Pro.

Guidelines:
1. Always be polite and empathetic
2. Use the FAQ document to answer questions
3. If you need to calculate (discounts, dates), use code_interpreter
4. If answer isn't in FAQ, be honest and offer to escalate
5. Keep responses concise but complete
6. End with "Is there anything else I can help with?"

Tone: Professional yet warm and friendly.""",
    function_list=['code_interpreter'],
    files=[os.path.abspath('product_faq.txt')]
)

print("✅ Created SmartWatch Support Bot\n")

In [ ]:
# Test with customer scenarios
customer_questions = [
    "Is the watch waterproof?",
    "I want to buy 3 watches at $299 each. Can you calculate my total with 15% discount?",
    "What's the return policy?",
    "Does it work with my Samsung Galaxy?"
]

for question in customer_questions:
    print("\n" + "="*70)
    print(f"Customer: {question}")
    print("="*70)
    
    messages = [{'role': 'user', 'content': question}]
    
    for response in support_bot.run(messages):
        for msg in response:
            if msg.get('function_call'):
                print(f"\n[Using tool: {msg['function_call']['name']}]")
            elif msg.get('content'):
                print(f"\nSupport: {msg['content']}")
    
    print()

---
## Part 7: Production Patterns

### Pattern 1: Error Handling

In [ ]:
def safe_assistant_call(bot, user_message, max_retries=3):
    """Call assistant with error handling and retries"""
    messages = [{'role': 'user', 'content': user_message}]
    
    for attempt in range(max_retries):
        try:
            responses = []
            for response in bot.run(messages):
                responses = response
            return responses
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt == max_retries - 1:
                return [{'role': 'assistant', 'content': f'Error: {str(e)}'}]
    
    return [{'role': 'assistant', 'content': 'Service unavailable'}]

# Test error handling
result = safe_assistant_call(support_bot, "What's the battery life?")
print(f"Result: {result[-1].get('content', '')[:100]}...")

### Pattern 2: Conversation Memory

In [ ]:
# Multi-turn conversation with memory
conversation_history = []

bot = Assistant(
    llm=llm_cfg,
    system_message='You are a helpful assistant. Remember context from previous messages.'
)

# Simulate multi-turn conversation
turns = [
    "My name is Alice",
    "I'm interested in learning Python",
    "What was my name again?"
]

for turn in turns:
    print(f"\nUser: {turn}")
    conversation_history.append({'role': 'user', 'content': turn})
    
    for response in bot.run(conversation_history):
        if response:
            assistant_msg = response[-1]
            print(f"Assistant: {assistant_msg.get('content', '')}")
            conversation_history.extend(response)

### Pattern 3: Streaming with typewriter_print

In [ ]:
from qwen_agent.utils.output_beautify import typewriter_print

bot = Assistant(llm=llm_cfg)
messages = [{'role': 'user', 'content': 'Write a haiku about coding'}]

print("\nStreaming response:\n")
response_text = ''
for response in bot.run(messages):
    response_text = typewriter_print(response, response_text)

---
## Part 8: Practice Exercises

### Exercise 1: Build a Code Review Bot

Create an assistant that:
- Reviews Python code for best practices
- Uses code_interpreter to test code
- Provides constructive feedback

In [ ]:
# TODO: Create code review assistant
# review_bot = Assistant(...)
# Test with some sample code

### Exercise 2: Technical Writer Assistant

Create an assistant that:
- Reads a code file
- Generates documentation
- Follows markdown format

In [ ]:
# TODO: Create technical writing assistant
# writer_bot = Assistant(...)
# Test with a sample code file

### Exercise 3: Multi-Tool Data Analyst

Create an assistant that:
- Uses code_interpreter for calculations
- Reads CSV/data files
- Provides insights and visualizations

In [ ]:
# TODO: Create data analyst assistant
# analyst_bot = Assistant(...)
# Test with sample data

---
## Summary: What You Learned

### Core Concepts Mastered

✅ **All Assistant parameters**
- `llm`: The AI brain
- `function_list`: Tools for capabilities
- `name` & `description`: Identity
- `system_message`: Behavior control
- `files`: Automatic RAG

✅ **System message engineering**
- Role-playing and personality
- Output formatting
- Expert personas
- Behavior rules

✅ **files parameter (RAG)**
- Automatic document processing
- Works with URLs and local files
- Instant Q&A over documents
- File-in-message pattern

✅ **function_list flexibility**
- String names for built-in tools
- Custom tool instances
- Mixed configurations
- MCP integration (Day 10)

✅ **Production patterns**
- Error handling with retries
- Conversation memory
- Streaming responses
- Multi-turn interactions

### Real-World Applications

1. **Customer Support** - FAQ + calculations + friendly tone
2. **HR Assistant** - Policy documents + specific citations
3. **Research Assistant** - Academic papers + technical explanations
4. **Code Helper** - Code execution + best practices
5. **Data Analyst** - File processing + insights

### Key Takeaways

1. **Assistant is production-ready** - Handles tools, files, errors automatically
2. **System messages are powerful** - Control personality, format, behavior
3. **files parameter = instant RAG** - No manual setup needed
4. **Mix and match tools** - Built-in + custom + MCP
5. **Conversation memory matters** - Maintain context for better UX

---

## What's Next?

**Tomorrow (Day 9): RAG Systems Deep Dive**

You'll learn:
- How RAG actually works under the hood
- ParallelDocQA for advanced document Q&A
- Chunking strategies and optimization
- When to use Assistant vs specialized RAG agents
- Performance tuning and scaling

---

**Congratulations! 🎉**

You've mastered the Assistant agent - the most versatile tool in Qwen-Agent!

You can now build production-ready AI assistants that:
- Use tools intelligently
- Learn from documents
- Follow your instructions
- Handle errors gracefully
- Maintain conversations

**You're ready for advanced topics!** 🚀